In [ ]:
import os
import glob
import shutil
import random
import xml.etree.ElementTree as ET
from PIL import Image
from ultralytics import YOLO
import torch

In [ ]:
print("INFO:\n")
print(f"Logical CPU threads available: {os.cpu_count()}")
print("PyTorch version:", torch.__version__)

cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)
if cuda_available:
    print("CUDA version:", torch.version.cuda)
    print("Device:", torch.cuda.get_device_name(0))


In [ ]:
IMAGE_DIR = "images"                 # Directory containing PNG images
XML_DIR = "annotations"              # Directory containing XML annotations
OUTPUT_DIR = "yolo_dataset"          # Directory to save the prepared dataset

# Create output directories
os.makedirs(os.path.join(OUTPUT_DIR, "images", "train"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "images", "val"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "labels", "train"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "labels", "val"), exist_ok=True)

In [ ]:
class_names = set()
for xml_file in glob.glob(os.path.join(XML_DIR, "*.xml")):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_names.add(class_name)

class_names = sorted(list(class_names))

print(f"Found {len(class_names)} unique brick classes: {class_names}")

In [ ]:
def xml_to_yolo_label(xml_file, image_width, image_height):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    yolo_annotations = []
    
    for obj in root.findall('object'):
        # Get class name
        class_name = obj.find('name').text
        
        # Get bounding box coordinates
        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)
        
        # Convert to YOLO format (normalized center_x, center_y, width, height)
        x_center = ((xmin + xmax) / 2) / image_width
        y_center = ((ymin + ymax) / 2) / image_height
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height
        
        # Format: class_id_index center_x center_y width height
        yolo_annotations.append(f"{class_names.index(class_name)} {x_center} {y_center} {width} {height}")
    
    return yolo_annotations

In [ ]:
# Get all image files
image_files = sorted(glob.glob(os.path.join(IMAGE_DIR, "*.png")))
print(f"Found {len(image_files)} image files")

# Randomly split into train and validation sets (80:20)
random.seed(42)  # for reproducibility
random.shuffle(image_files)
split_idx = int(0.8 * len(image_files))
train_images = image_files[:split_idx]
test_images = image_files[split_idx:]

print(f"Training images: {len(train_images)}")
print(f"Validation images: {len(test_images)}")

# Process the images and annotations
def process_dataset(image_files, dataset_type):
    for img_path in image_files:

        # Get image filename without extension
        img_filename = os.path.basename(img_path)
        img_name = os.path.splitext(img_filename)[0]
        
        # Load the image to get dimensions
        img = Image.open(img_path)
        img_width, img_height = img.size
        
        # Find the corresponding XML file
        xml_path = os.path.join(XML_DIR, f"{img_name}.xml")
        
        if os.path.exists(xml_path):
            
            # Copy image to dataset directory
            output_img_path = os.path.join(OUTPUT_DIR, "images", dataset_type, img_filename)
            shutil.copy(img_path, output_img_path)

            # Convert annotations to YOLO labels
            yolo_annotations = xml_to_yolo_label(xml_path, img_width, img_height)
            
            # Save YOLO annotations
            output_label_path = os.path.join(OUTPUT_DIR, "labels", dataset_type, f"{img_name}.txt")
            with open(output_label_path, "w") as f:
                f.write("\n".join(yolo_annotations))
        else:
            print(f"Warning: XML file not found for {img_path}")

# Process training and validation sets
process_dataset(train_images, "train")
process_dataset(test_images, "val")

In [ ]:
# Create dataset.yaml file
yaml_content = f"""
# YOLO dataset configuration
path: {os.path.abspath(OUTPUT_DIR)}
train: images/train
val: images/val

# Classes
nc: {len(class_names)}
names: {class_names}
"""

with open(os.path.join(OUTPUT_DIR, "dataset.yaml"), "w") as f:
    f.write(yaml_content)

print("Dataset preparation complete!")

In [ ]:
# Select base model
model = YOLO('yolov8m.pt')

# Clear cache
if cuda_available:
    torch.cuda.empty_cache()

# Start training
results = model.train(
    data=os.path.join(OUTPUT_DIR, "dataset.yaml"),
    epochs=500,
    imgsz=1024,
    batch=-1,
    patience=20,
    device='cuda',
    project=OUTPUT_DIR,
    name='lego_yolo8m'
)

In [ ]:
metrics = model.val(data=os.path.join(OUTPUT_DIR, "dataset.yaml"), split='val') 

# Show key metrics
print(f"mAP@50-95: {metrics.box.map:.3f}")
print(f"mAP@50: {metrics.box.map50:.3f}")
print(f"mAP@75: {metrics.box.map75:.3f}")